In [1]:
# no figures made here but these scripts check significance of pathways in Fig. ED5k,l

# python version 3.6.9 #
import pandas as pd # 1.1.5
import numpy as np # 1.19.5
import matplotlib.pyplot as plt
import matplotlib # 3.3.4
import scipy.stats as st # scipy 1.5.4
import random

matplotlib.rcParams['font.sans-serif'] = "Arial"
matplotlib.rcParams['font.family'] = "sans-serif"
plt.rc('text', usetex=False)
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
matplotlib.rcParams.update({'font.size': 16})
plt.style.use('source_data/included/figure.style')
cm = plt.cm.get_cmap('Accent')
from scipy.stats import hypergeom
import seaborn as sns
from scipy.spatial import distance
from scipy.cluster import hierarchy
import scipy


In [2]:
## read in differential expression vs. stationary (cluster 1) and save for ipage
## files generated in v4_markers_vs_clust1.R

FDR = 0.01
bnum_table = pd.read_csv('source_data/included/ecoli_bnum_table_updated_2.txt',sep='\t',index_col=0)
bnum_table = bnum_table.drop_duplicates()
bnum_table.index = bnum_table['gene']

def make_table(filename,FDR):
    table = pd.read_csv(filename,sep='\t',index_col=0)
    table.index = table['Row.names']
    table = table.drop('Row.names',axis=1)
    table = table.merge(bnum_table,left_index=True,right_index=True)
    table['less'] = -table['less']
    signed_p = []
    for h,l in zip(table['greater'],table['less']):
        if abs(h)<=abs(l):
            signed_p.append(h + (10**-280))
        elif abs(l)<abs(h):
            signed_p.append(l - (10**-280))
        else:
            print('error')
    table['signed_p'] = signed_p
    table['fixed_signif'] = (abs(table['signed_p']).sort_values()<FDR*np.arange(1,len(table)+1)/len(table)).astype(int)
    table['fixed_signif'] = (table['fixed_signif'] * (table['signed_p']/abs(table['signed_p']))).astype(int)
    return table

tables = {}
tables['metG_clust2'] = make_table('source_data/generated/v4_metG_clust2_vs_1.txt',FDR)
tables['d6_clust2'] = make_table('source_data/generated/v4_d6_clust2_vs_1.txt',FDR)
tables['hipA7_clust2'] = make_table('source_data/generated/v4_hipA7_clust2_vs_1.txt',FDR)
tables['tet'] = make_table('source_data/generated/v4_tet_vs_1.txt',FDR)

counts = {}
counts['metG_clust2'] = pd.read_csv('source_data/generated/v4_metG_clust2_vs_1_counts.txt',sep='\t')
counts['d6_clust2'] = pd.read_csv('source_data/generated/v4_d6_clust2_vs_1_counts.txt',sep='\t')
counts['hipA7_clust2'] = pd.read_csv('source_data/generated/v4_hipA7_clust2_vs_1_counts.txt',sep='\t')
counts['tet'] = pd.read_csv('source_data/generated/v4_tet_vs_1_counts.txt',sep='\t')

s1 = 'metG_clust2'
df = tables[s1]
df = df.merge(counts[s1].loc[counts[s1].min(axis=1)>0],how='inner',left_index=True,right_index=True).drop(['bulk.1','bulk.2'],axis=1)
archive_df = df.copy()
pd.DataFrame(archive_df['fixed_signif'].values+1,index=archive_df['bnum']).sort_values(0).to_csv('source_data/generated/' + s1 + '_FDR0.01_discrete_vs_1.txt',sep='\t')

s1 = 'd6_clust2'
df = tables[s1]
df = df.merge(counts[s1].loc[counts[s1].min(axis=1)>0],how='inner',left_index=True,right_index=True).drop(['bulk.1','bulk.2'],axis=1)
archive_df = df.copy()
pd.DataFrame(archive_df['fixed_signif'].values+1,index=archive_df['bnum']).sort_values(0).to_csv('source_data/generated/' + s1 + '_FDR0.01_discrete_vs_1.txt',sep='\t')

s1 = 'hipA7_clust2'
df = tables[s1]
df = df.merge(counts[s1].loc[counts[s1].min(axis=1)>0],how='inner',left_index=True,right_index=True).drop(['bulk.1','bulk.2'],axis=1)
archive_df = df.copy()
pd.DataFrame(archive_df['fixed_signif'].values+1,index=archive_df['bnum']).sort_values(0).to_csv('source_data/generated/' + s1 + '_FDR0.01_discrete_vs_1.txt',sep='\t')

s1 = 'tet'
df = tables[s1]
df = df.merge(counts[s1].loc[counts[s1].min(axis=1)>0],how='inner',left_index=True,right_index=True).drop(['bulk.1','bulk.2'],axis=1)
archive_df = df.copy()
pd.DataFrame(archive_df['fixed_signif'].values+1,index=archive_df['bnum']).sort_values(0).to_csv('source_data/generated/' + s1 + '_FDR0.01_discrete_vs_1.txt',sep='\t')


In [3]:
## read in differential expression vs. early exponential (cluster 4) and save for ipage
## files generated in v4_markers_vs_clust4.R

tables = {}
FDR = 0.01
tables['metG_clust2'] = make_table('source_data/generated/v4_metG_clust2_vs_4.txt',FDR)
tables['d6_clust2'] = make_table('source_data/generated/v4_d6_clust2_vs_4.txt',FDR)
tables['hipA7_clust2'] = make_table('source_data/generated/v4_hipA7_clust2_vs_4.txt',FDR)
tables['tet'] = make_table('source_data/generated/v4_tet_vs_4.txt',FDR)

counts = {}
counts['metG_clust2'] = pd.read_csv('source_data/generated/v4_metG_clust2_vs_4_counts.txt',sep='\t')
counts['d6_clust2'] = pd.read_csv('source_data/generated/v4_d6_clust2_vs_4_counts.txt',sep='\t')
counts['hipA7_clust2'] = pd.read_csv('source_data/generated/v4_hipA7_clust2_vs_4_counts.txt',sep='\t')
counts['tet'] = pd.read_csv('source_data/generated/v4_tet_vs_4_counts.txt',sep='\t')

s1 = 'metG_clust2'
df = tables[s1]
df = df.merge(counts[s1].loc[counts[s1].min(axis=1)>0],how='inner',left_index=True,right_index=True).drop(['bulk.1','bulk.2'],axis=1)
archive_df = df.copy()
pd.DataFrame(archive_df['fixed_signif'].values+1,index=archive_df['bnum']).sort_values(0).to_csv('source_data/generated/' + s1 + '_FDR0.01_discrete_vs_4.txt',sep='\t')

s1 = 'd6_clust2'
df = tables[s1]
df = df.merge(counts[s1].loc[counts[s1].min(axis=1)>0],how='inner',left_index=True,right_index=True).drop(['bulk.1','bulk.2'],axis=1)
archive_df = df.copy()
pd.DataFrame(archive_df['fixed_signif'].values+1,index=archive_df['bnum']).sort_values(0).to_csv('source_data/generated/' + s1 + '_FDR0.01_discrete_vs_4.txt',sep='\t')

s1 = 'hipA7_clust2'
df = tables[s1]
df = df.merge(counts[s1].loc[counts[s1].min(axis=1)>0],how='inner',left_index=True,right_index=True).drop(['bulk.1','bulk.2'],axis=1)
archive_df = df.copy()
pd.DataFrame(archive_df['fixed_signif'].values+1,index=archive_df['bnum']).sort_values(0).to_csv('source_data/generated/' + s1 + '_FDR0.01_discrete_vs_4.txt',sep='\t')

s1 = 'tet'
df = tables[s1]
df = df.merge(counts[s1].loc[counts[s1].min(axis=1)>0],how='inner',left_index=True,right_index=True).drop(['bulk.1','bulk.2'],axis=1)
archive_df = df.copy()
pd.DataFrame(archive_df['fixed_signif'].values+1,index=archive_df['bnum']).sort_values(0).to_csv('source_data/generated/' + s1 + '_FDR0.01_discrete_vs_4.txt',sep='\t')


In [4]:
## run ipage discrete mode, P.1 ##
## read in aggregated iPAGE output ## 
combined = pd.read_csv('source_data/included/discrete_vs_1_pvmatrix_combined.txt',sep='\t',index_col=0)
## clean up dataframe and make parsable ##
combined['comparison'] = combined.index.str.split('_FDR0.01_discrete',expand=True).droplevel(1)
combined['type'] = combined.index.str.split('_PO_',expand=True).droplevel(0).str.split('_P.1_PAGE',expand=True).droplevel(1)
combined = combined.reset_index().drop('index',axis=1)
for column in ['0','1','2']:
    combined = combined.merge(combined[column].str.split('/',expand=True).rename(columns={0:column+'_pval_enriched',1:column+'_pval_depleted'}).astype(float),left_index=True,right_index=True)
combined = combined.drop(['0','1','2'],axis=1)
## resulting combined dataframe gives p-value (from iPAGE) of enrichment or depletion in each gene classification ##
## gene classifications are 0 (significantly underexpressed), 1 (not differentially expressed), 2 (significantly overexpressed) ##
combined_vs_1 = combined.copy()



In [5]:
## read in aggregated iPAGE output ## 
## run ipage P.1 ##
combined = pd.read_csv('source_data/included/discrete_vs_4_pvmatrix_combined.txt',sep='\t',index_col=0)
## clean up dataframe and make parsable ##
combined['comparison'] = combined.index.str.split('_FDR0.01_discrete',expand=True).droplevel(1)
combined['type'] = combined.index.str.split('_PO_',expand=True).droplevel(0).str.split('_P.1_PAGE',expand=True).droplevel(1)
combined = combined.reset_index().drop('index',axis=1)
for column in ['0','1','2']:
    combined = combined.merge(combined[column].str.split('/',expand=True).rename(columns={0:column+'_pval_enriched',1:column+'_pval_depleted'}).astype(float),left_index=True,right_index=True)
combined = combined.drop(['0','1','2'],axis=1)
## resulting combined dataframe gives p-value (from iPAGE) of enrichment or depletion in each gene classification ##
## gene classifications are 0 (significantly underexpressed), 1 (not differentially expressed), 2 (significantly overexpressed) ##
combined_vs_4 = combined.copy()



In [6]:
### find pathways below threshold
p_threshold = np.log10(0.05)

In [7]:
combined_vs_4.loc[(combined_vs_4['GO'].str.contains('PspF')) & (combined_vs_4['2_pval_enriched']<p_threshold)]


,GO,comparison,type,0_pval_enriched,0_pval_depleted,1_pval_enriched,1_pval_depleted,2_pval_enriched,2_pval_depleted
845,PspF_Up PspF_Up,hipA7_clust2,tf,0.0,-0.093,0.0,-4.492,-5.059,-0.0
919,PspF_Up PspF_Up,tet,tf,0.0,-0.395,0.0,-3.600,-5.737,0.0
980,PspF_Up PspF_Up,metG_clust2,tf,0.0,-0.534,-0.0,-1.828,-3.048,0.0


In [8]:
combined_vs_1.loc[(combined_vs_1['GO'].str.contains('PspF')) & (combined_vs_1['2_pval_enriched']<p_threshold)]


,GO,comparison,type,0_pval_enriched,0_pval_depleted,1_pval_enriched,1_pval_depleted,2_pval_enriched,2_pval_depleted
969,PspF_Up PspF_Up,hipA7_clust2,tf,0.0,-0.210,-0.001,-1.592,-2.266,-0.0
1015,PspF_Up PspF_Up,tet,tf,-0.0,-0.701,-0.000,-2.551,-5.124,-0.0
1087,PspF_Up PspF_Up,metG_clust2,tf,-0.0,-0.767,-0.000,-2.253,-4.676,0.0


In [9]:
combined_vs_1.loc[(combined_vs_1['GO'].str.contains('oligopeptide transporter')) & (combined_vs_1['2_pval_enriched']<p_threshold)]



,GO,comparison,type,0_pval_enriched,0_pval_depleted,1_pval_enriched,1_pval_depleted,2_pval_enriched,2_pval_depleted
244,GO:0015198 oligopeptide transporter activity,d6_clust2,go,0.0,-0.609,-0.001,-2.1,-3.417,-0.0


In [10]:
combined_vs_4.loc[(combined_vs_4['GO'].str.contains('oligopeptide transporter')) & (combined_vs_4['2_pval_enriched']<p_threshold)]


,GO,comparison,type,0_pval_enriched,0_pval_depleted,1_pval_enriched,1_pval_depleted,2_pval_enriched,2_pval_depleted
168,GO:0015198 oligopeptide transporter activity,d6_clust2,go,-0.0,-0.343,-0.029,-0.698,-1.379,-0.003


In [11]:
combined_vs_1.loc[(combined_vs_1['GO'].str.contains('Fur_Down')) & (combined_vs_1['2_pval_enriched']<p_threshold)]



,GO,comparison,type,0_pval_enriched,0_pval_depleted,1_pval_enriched,1_pval_depleted,2_pval_enriched,2_pval_depleted
1011,Fur_Down Fur_Down,tet,tf,-0.018,-1.099,-0.000,-4.495,-10.095,0.0
1168,Fur_Down Fur_Down,d6_clust2,tf,-0.022,-0.921,-0.001,-2.303,-4.794,-0.0


In [12]:
combined_vs_4.loc[(combined_vs_4['GO'].str.contains('Fur_Down')) & (combined_vs_4['2_pval_enriched']<p_threshold)]


,GO,comparison,type,0_pval_enriched,0_pval_depleted,1_pval_enriched,1_pval_depleted,2_pval_enriched,2_pval_depleted
939,Fur_Down Fur_Down,tet,tf,-0.389,-0.141,-0.002,-2.059,-2.48,-0.0


In [13]:
combined_vs_1.loc[(combined_vs_1['GO'].str.contains('OxyR')) & (combined_vs_1['2_pval_enriched']<p_threshold)]


,GO,comparison,type,0_pval_enriched,0_pval_depleted,1_pval_enriched,1_pval_depleted,2_pval_enriched,2_pval_depleted


In [14]:
combined_vs_4.loc[(combined_vs_4['GO'].str.contains('OxyR')) & (combined_vs_4['2_pval_enriched']<p_threshold)]


,GO,comparison,type,0_pval_enriched,0_pval_depleted,1_pval_enriched,1_pval_depleted,2_pval_enriched,2_pval_depleted
923,OxyR_Up OxyR_Up,tet,tf,0.0,-0.99,-0.002,-1.749,-4.166,-0.0
